In [3]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [136]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from random import seed, shuffle
import os,sys
import traceback
import numpy as np
from random import seed, shuffle
from collections import defaultdict
from copy import deepcopy
from cvxpy import *
import utils as ut
import cvxpy as cp
import dccp
from dccp.problem import is_dccp

In [98]:
df=pd.read_csv('/content/drive/MyDrive/compas-scores-two-years.csv')
# data preprocessing
df = df[df['race'].isin(['Caucasian', 'African-American'])]
data = df[['age', 'sex', 'race', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'c_charge_degree', 'priors_count','is_recid', 'is_violent_recid', 'two_year_recid']]

# convert categorical data to numerical
data['c_charge_degree'] = data['c_charge_degree'].astype('category').cat.codes
data['race'] = data['race'].astype('category').cat.codes
data['sex'] = data['sex'].astype('category').cat.codes

# set to features and target
X = data.drop(['race','two_year_recid'], axis=1)
X = X.fillna(0)
y = data['two_year_recid']
y[y==0] = -1 #convert label to 1 and -1
Z = data['race']

# standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# split data to training, validating and testing sets
X_train, X_test, y_train, y_test, Z_train, Z_test = train_test_split(X, y, Z, test_size=0.3, random_state=6)

<ipython-input-98-213d93daafdf>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['c_charge_degree'] = data['c_charge_degree'].astype('category').cat.codes
<ipython-input-98-213d93daafdf>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['race'] = data['race'].astype('category').cat.codes
<ipython-input-98-213d93daafdf>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [26]:

Z_train == 0

7183     True
3735    False
6235    False
6580     True
33      False
        ...  
400      True
5520     True
2371     True
274      True
3260    False
Name: race, Length: 4305, dtype: bool

In [68]:
def get_constraint(x_train, y_train, Z_train, c, w):
    constraints = []
    N = len(Z_train)
    N1 = np.sum(Z_train)
    N0 = N-N1

    idx0 = Z_train == 0
    idx1 = Z_train == 1

    d0 = cp.multiply(y_train[idx0], x_train[idx0] * w) # y.f(x)
    d1 = cp.multiply(y_train[idx1], x_train[idx1] * w) # y.f(x)

    g00 = cp.sum(cp.minimum(0, cp.multiply((1 - y_train[idx0])/2.0, d0)))
    g01 = cp.sum(cp.minimum(0, cp.multiply((1 - y_train[idx1])/2.0, d1)))

    g10 = cp.sum(cp.minimum(0, cp.multiply((1 + y_train[idx0])/2.0, d0)))
    g11 = cp.sum(cp.minimum(0, cp.multiply((1 + y_train[idx1])/2.0, d1)))

    constraints.append( (-N1/N)*g01 + (N0/N)*g00 <= c)
    constraints.append( (-N1/N)*g01 + (N0/N)*g00 >= -c)
    constraints.append( (-N1/N)*g11 + (N0/N)*g10 <= c)
    constraints.append( (-N1/N)*g11 + (N0/N)*g10 >= -c)
            

    return constraints   

In [265]:
def train_model(x, y, z, c):

    max_iters = 100 
    max_iter_dccp = 50  

    num_points, num_features = x.shape

    w = cp.Variable(x.shape[1])
    b = cp.Variable()

    loss = cp.sum(cp.pos(1 - cp.multiply(y, x @ w + b)))

    constraints = []

    N = len(z)
    N1 = np.sum(z)
    N0 = N-N1

    idx0 = z == 0
    idx1 = z == 1

    g00 = (-N1/N)*cp.sum(cp.minimum(0, cp.multiply(y[idx0], (x[idx0] @ w + b))))
    g01 = (N0/N)*cp.sum(cp.minimum(0, cp.multiply(y[idx1], (x[idx1] @ w + b))))

    constraints.append( g00 <= -g01 + c)
    constraints.append( g00 >= -g01 - c)

    prob = cp.Problem(cp.Minimize(loss), constraints)
    
    try:


        prob.solve(method='dccp')


    except:
        traceback.print_exc()
        sys.stdout.flush()
        sys.exit(1)

        

    w = np.array(w.value).flatten() # flatten converts it to a 1d array
    
    return w, b.value

In [272]:
def classification(c):
    
    w,b = train_model(X_train, y_train, Z_train, c)

    db_train = np.dot(X_train, w) + b
    y_hat_train = np.sign(db_train)

    db_test = np.dot(X_test, w) + b
    y_hat_test = np.sign(db_test)

    idx0 = Z_train == 0
    idx1 = Z_train == 1



    correct_train = (y_hat_train == y_train).astype(int) 
    train_score = (np.sum(correct_train)) / (len(correct_train))

    acc_dif_train = np.abs(np.sum(correct_train[idx0])/len(idx0)-np.sum(correct_train[idx1])/len(idx1))
    
    idx0 = Z_test == 0
    idx1 = Z_test == 1
    
    correct_test = (y_hat_test == y_test).astype(int)
    print(np.sum(correct_test))
    test_score = (np.sum(correct_test)) / (len(correct_test))

    acc_dif_test = np.abs(np.sum(correct_test[idx0])/len(idx0)-np.sum(correct_test[idx1])/len(idx1))

  
    
    
    return train_score, test_score, acc_dif_train, acc_dif_test

In [158]:
c = 0.05

In [273]:
train_acc, test_acc, acc_dif_train, acc_dif_test = classification(c)
print("Training accuracy:", train_acc)
print("Test accuracy:", test_acc)
print("Calibration train:", acc_dif_train)
print("Calibration test:", acc_dif_test)

1777
Training accuracy: 0.9616724738675958
Test accuracy: 0.9631436314363143
Calibration train: 0.18490127758420438
Calibration test: 0.1869918699186992
